# TMDB 5000 Movie Dataset
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata/data

## 引入套件

In [1]:
import requests
import zipfile
import os
import shutil
import pandas as pd
import json

## 獲得原始資料

In [2]:
url = "https://www.kaggle.com/api/v1/datasets/download/tmdb/tmdb-movie-metadata"
zip_path = "./archive.zip"
response = requests.get(url, allow_redirects=True)
with open(zip_path, 'wb') as file:
    file.write(response.content)
print("Download completed!")

Download completed!


In [3]:
# 指定 ZIP 文件的路徑
extract_path = "./raw"  # 解壓縮的目標目錄

# 確保目標目錄存在，否則創建它
os.makedirs(extract_path, exist_ok=True)

# 解壓縮 ZIP 文件
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction completed!")


Extraction completed!


## 處理tmdb_5000_credits.csv

In [4]:
# 讀取csv存成credits變數
credits = pd.read_csv(extract_path+'/tmdb_5000_credits.csv')
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
# 讀取credits中的每一筆資料
# 解析cast和crew的json格式
# 分別放入casts_array和crews_array
casts_array = []
crews_array = []
for _, credit in credits.iterrows():
    movie_id = credit["movie_id"]
    # casts
    casts_json = json.loads(credit["cast"])
    for cast in casts_json:
        casts_array.append([movie_id,cast["cast_id"],cast["character"],cast["credit_id"],cast["gender"],cast["id"],cast["name"],cast["order"]])
    
    # crews
    crews_json = json.loads(credit["crew"])
    for crew in crews_json:
        crews_array.append([movie_id,crew["credit_id"],crew["department"],crew["gender"],crew["id"],crew["job"],crew["name"]])

In [6]:
# 把casts_array和crews_array轉成dataframe
casts = pd.DataFrame(casts_array,columns=["movie_id","cast_id","character","credit_id","gender","actor_id","name","order"])
crews = pd.DataFrame(crews_array,columns=["movie_id","credit_id","department","gender","worker_id","job","name"])

### 處理casts

In [7]:
# 從casts中取出id,name,gender存入actors
# 把gender從數值轉換成Male / Female
# 存成actors.csv
actors = casts[["actor_id","name","gender"]]
actors['gender'] = actors['gender'].replace({0: 'Male', 1: 'Male', 2: 'Female'})
actors = actors.drop_duplicates(subset=['actor_id', 'name','gender'])
actors.to_csv("actors.csv",index=False)
actors.head()

/var/folders/l3/l158zncd2b5508v6161klssw0000gn/T/ipykernel_47204/2697906575.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actors['gender'] = actors['gender'].replace({0: 'Male', 1: 'Male', 2: 'Female'})


,actor_id,name,gender
0,65731,Sam Worthington,Female
1,8691,Zoe Saldana,Male
2,10205,Sigourney Weaver,Male
3,32747,Stephen Lang,Female
4,17647,Michelle Rodriguez,Male


In [8]:
# 因為gender和name已經移到actors，且cast_id不知用途，移除gender,name,cast_id三個欄位
# 把casts存成casts.csv
casts = casts.drop(["cast_id","gender","name"],axis=1)
casts.to_csv("casts.csv",index=False)
casts.head()

,movie_id,character,credit_id,actor_id,order
0,19995,Jake Sully,5602a8a7c3a3685532001c9a,65731,0
1,19995,Neytiri,52fe48009251416c750ac9cb,8691,1
2,19995,Dr. Grace Augustine,52fe48009251416c750aca39,10205,2
3,19995,Col. Quaritch,52fe48009251416c750ac9cf,32747,3
4,19995,Trudy Chacon,52fe48009251416c750ac9d3,17647,4


### 處理crews

In [9]:
# 從crews中取出id,name,gender存入workers
# 把gender從數值轉換成Male / Female
# 存成workers.csv
workers = crews[["worker_id","name","gender"]]
workers['gender'] = workers['gender'].replace({0: 'Male', 1: 'Male', 2: 'Female'})
workers = workers.drop_duplicates(subset=['worker_id', 'name','gender'])
workers.to_csv("workers.csv",index=False)
workers.head()

/var/folders/l3/l158zncd2b5508v6161klssw0000gn/T/ipykernel_47204/868642444.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  workers['gender'] = workers['gender'].replace({0: 'Male', 1: 'Male', 2: 'Female'})


,worker_id,name,gender
0,1721,Stephen E. Rivkin,Male
1,496,Rick Carter,Female
2,900,Christopher Boyes,Male
4,1262,Mali Finn,Male
5,1729,James Horner,Female


In [10]:
# 因為gender和name已經移到workers，移除gender,name兩個欄位
# 把crews存成crews.csv
crews = crews.drop(["gender","name"],axis=1)
crews.to_csv("crews.csv",index=False)
crews.head()

,movie_id,credit_id,department,worker_id,job
0,19995,52fe48009251416c750aca23,Editing,1721,Editor
1,19995,539c47ecc3a36810e3001f87,Art,496,Production Design
2,19995,54491c89c3a3680fb4001cf7,Sound,900,Sound Designer
3,19995,54491cb70e0a267480001bd0,Sound,900,Supervising Sound Editor
4,19995,539c4a4cc3a36810c9002101,Production,1262,Casting


## 處理tmdb_5000_movies.csv

In [11]:
# 讀取csv存成movies變數
movies = pd.read_csv(extract_path+'/tmdb_5000_movies.csv')
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [12]:
# 讀取movies中的每一筆資料
# 解析genres,keywords,production_companies,production_countries,spoken_languages的json格式
# 分別放入xxx_array
genres_array = []
keywords_array = []
production_companies_array = []
production_countries_array = []
spoken_languages_array = []
for _, movie in movies.iterrows():
    movie_id = movie["id"]
    # genres
    genres_json = json.loads(movie["genres"])
    for i in genres_json:
        genres_array.append([movie_id,i["id"],i["name"]])

    # keywords
    keywords_json = json.loads(movie["keywords"])
    for i in keywords_json:
        keywords_array.append([movie_id,i["id"],i["name"]])

    # production_companies
    production_companies_json = json.loads(movie["production_companies"])
    for i in production_companies_json:      
        production_companies_array.append([movie_id,i["id"],i["name"]])

    # production_companies
    production_countries_json = json.loads(movie["production_countries"])
    for i in production_countries_json:      
        production_countries_array.append([movie_id,i["iso_3166_1"],i["name"]])

    # spoken_languages
    spoken_languages_json = json.loads(movie["spoken_languages"])
    for i in spoken_languages_json:      
        spoken_languages_array.append([movie_id,i["iso_639_1"],i["name"]])

In [13]:
# 把xxx_array和crews_array轉成dataframe
genres = pd.DataFrame(genres_array,columns=["movie_id","genre_id","name"])
keywords = pd.DataFrame(keywords_array,columns=["movie_id","keyword_id","name"])
production_companies = pd.DataFrame(production_companies_array,columns=["movie_id","production_company_id","name"])
production_countries = pd.DataFrame(production_countries_array,columns=["movie_id","iso_3166_1","name"])
spoken_languages = pd.DataFrame(spoken_languages_array,columns=["movie_id","iso_639_1","name"])

### 處理genres

In [14]:
# 從genres中取出genre_id,name存入genres_type
# 存成genres_type.csv
genres_type = genres[["genre_id","name"]]
genres_type = genres_type.drop_duplicates(subset=["genre_id","name"])
genres_type.to_csv("genres_type.csv",index=False)
genres_type.head()

,genre_id,name
0,28,Action
1,12,Adventure
2,14,Fantasy
3,878,Science Fiction
9,80,Crime


In [15]:
# 因為name已經移到genres_type，移除name欄位
# 把genres存成genres.csv
genres = genres.drop(["name"],axis=1)
genres.to_csv("genres.csv",index=False)
genres.head()

,movie_id,genre_id
0,19995,28
1,19995,12
2,19995,14
3,19995,878
4,285,12


### 處理keywords

In [16]:
# 從keywords中取出keyword_id,name存入keyword_type
# 存成keyword_type.csv
keyword_type = keywords[["keyword_id","name"]]
keyword_type = keyword_type.drop_duplicates(subset=["keyword_id","name"])
keyword_type.to_csv("keyword_type.csv",index=False)
keyword_type.head()

,keyword_id,name
0,1463,culture clash
1,2964,future
2,3386,space war
3,3388,space colony
4,3679,society


In [17]:
# 因為name已經移到keyword_type，移除name欄位
# 把keywords存成keywords.csv
keywords = keywords.drop(["name"],axis=1)
keywords.to_csv("keywords.csv",index=False)
keywords.head()

,movie_id,keyword_id
0,19995,1463
1,19995,2964
2,19995,3386
3,19995,3388
4,19995,3679


### 處理production_companies

In [18]:
# 從production_companies中取出production_company_id,name存入production_company_type
# 存成production_company_type.csv
production_company_type = production_companies[["production_company_id","name"]]
production_company_type = production_company_type.drop_duplicates(subset=["production_company_id","name"])
production_company_type.to_csv("production_company_type.csv",index=False)
production_company_type.head()

,production_company_id,name
0,289,Ingenious Film Partners
1,306,Twentieth Century Fox Film Corporation
2,444,Dune Entertainment
3,574,Lightstorm Entertainment
4,2,Walt Disney Pictures


In [19]:
# 因為name已經移到production_company_type，移除name欄位
# 把production_companies存成production_companies.csv
production_companies = production_companies.drop(["name"],axis=1)
production_companies.to_csv("production_companies.csv",index=False)
production_companies.head()

,movie_id,production_company_id
0,19995,289
1,19995,306
2,19995,444
3,19995,574
4,285,2


### 處理production_countries

In [20]:
# 從production_countries中取出iso_3166_1,name存入production_country_type
# 存成production_country_type.csv
production_country_type = production_countries[["iso_3166_1","name"]]
production_country_type = production_country_type.drop_duplicates(subset=["iso_3166_1","name"])
production_country_type.to_csv("production_country_type.csv",index=False)
production_country_type.head()

,iso_3166_1,name
0,US,United States of America
1,GB,United Kingdom
16,JM,Jamaica
18,BS,Bahamas
19,DM,Dominica


In [21]:
# 因為name已經移到production_country_type，移除name欄位
# 把production_countries存成production_countries.csv
production_countries = production_countries.drop(["name"],axis=1)
production_countries.to_csv("production_countries.csv",index=False)
production_countries.head()

,movie_id,iso_3166_1
0,19995,US
1,19995,GB
2,285,US
3,206647,GB
4,206647,US


### 處理spoken_languages

In [22]:
# 從spoken_languages中取出iso_639_1,name存入spoken_language_type
# 存成spoken_language_type.csv
spoken_language_type = spoken_languages[["iso_639_1","name"]]
spoken_language_type = spoken_language_type.drop_duplicates(subset=["iso_639_1","name"])
spoken_language_type.to_csv("spoken_language_type.csv",index=False)
spoken_language_type.head()

,iso_639_1,name
0,en,English
1,es,Español
3,fr,Français
6,it,Italiano
7,de,Deutsch


In [23]:
# 因為name已經移到spoken_language_type，移除name欄位
# 把spoken_languages存成spoken_languages.csv
spoken_languages = spoken_languages.drop(["name"],axis=1)
spoken_languages.to_csv("spoken_languages.csv",index=False)
spoken_languages.head()

,movie_id,iso_639_1
0,19995,en
1,19995,es
2,285,en
3,206647,fr
4,206647,en


### 處理Movies

In [24]:
# 刪除movies中已經處理完json的欄位後存成movies.csv
movies = movies.drop(["genres","keywords","production_companies","production_countries","spoken_languages"],axis=1)
movies.to_csv("movies.csv",index=False)
movies.head()

,budget,homepage,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count
0,237000000,http://www.avatarmovie.com/,19995,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,http://disney.go.com/disneypictures/pirates/,285,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,http://www.sonypictures.com/movies/spectre/,206647,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,880674609,148.0,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,http://www.thedarkknightrises.com/,49026,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,1084939099,165.0,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,http://movies.disney.com/john-carter,49529,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,284139100,132.0,Released,"Lost in our world, found in another.",John Carter,6.1,2124


## 刪除原始檔案

In [25]:
os.remove(zip_path)
shutil.rmtree(extract_path)

## 驗證

In [28]:
def unique_verify(df,groupby_col,unique_col):
    # 找出具有相同 groupby_col 但不同 unique_col 的行
    duplicate_ids = df.groupby(groupby_col)[unique_col].nunique()
    # 篩選出具有多個不同 name 的 id
    duplicate_ids = duplicate_ids[duplicate_ids > 1]
    return duplicate_ids

In [30]:
unique_verify(actors,"actor_id","gender")

Series([], Name: gender, dtype: int64)